In [4]:
import os
import fnmatch
import hashlib
from nbconvert import PythonExporter

In [5]:
exporter = PythonExporter()
exporter.exclude_input_prompt = True

# Define the resources dictionary with a 'metadata' key
exp_resources = {'no_promt': True}

In [9]:
def get_package_name(package_path):
    folder_name = os.path.basename(package_path)
    if os.path.isfile(os.path.join(package_path, "__init__.py")):
        return folder_name
    return None

def process_python_file(project_name, package_name, parent_folder_name, file_path, source=None):
     # Compute the MD5 checksum for the file
    with open(file_path, "rb") as f:
        file_contents = f.read()
        file_checksum = hashlib.md5(file_contents).hexdigest()
        
    # check if file path in database
    
    # check if file checksum matches checksum in database
    
    # if not, process the file
    
    # Your processing code here
    print(f"Project: {project_name}: Processing {package_name} in folder {parent_folder_name}: {file_path} - {source} - {file_checksum}")
    
def process_other_file(project_name, package_name, parent_folder_name, file_path):
    # Your processing code here
    print(f"Project: {project_name}: Processing {package_name} in folder {parent_folder_name}: {file_path}")

def process_file_factory(folder_path, file_name, project_name, package_name, parent_folder_name, included_file_types=["Dockerfile", "*.toml", "*.md", "*.cfg"]):
    if file_name.endswith(".py"):
        file_path = os.path.join(folder_path, file_name)
        process_python_file(project_name, package_name, parent_folder_name, file_path)
    elif file_name.endswith(".ipynb"):
        file_path = os.path.join(folder_path, file_name)
        with open(file_path) as f:
            source, resources = exporter.from_file(f, resources=exp_resources)
            print(resources)
    for pattern in included_file_types:
        if fnmatch.fnmatch(file_name, pattern):
            process_other_file(project_name, package_name, parent_folder_name, os.path.join(folder_path, file_name))
            break
        
    

In [10]:
def process_python_files(top_package_name, root_dir, package_name=None, exclude_folders=[]):
    #if package_name is None:
    #    package_name = os.path.basename(root_dir)
    subfolders = []
    src_dir = os.path.join(root_dir, "src")
    if os.path.isdir(src_dir):
        subfolders.append(src_dir)
        
        # set top package name to parent of src folder
        if top_package_name is None:
            top_package_name = root_dir
            
    tests_dir = os.path.join(root_dir, "tests")
    if os.path.isdir(tests_dir):
        subfolders.append(tests_dir)
    notebooks_dir = os.path.join(root_dir, "notebooks")
    if os.path.isdir(notebooks_dir):
        subfolders.append(notebooks_dir)
        
    subfolders += [os.path.join(root_dir, f) for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f)) and f not in ["src", "tests", "notebooks"] and f not in exclude_folders]
    for folder_path in subfolders:
        # a or b returns a if a is True, else returns b.
        # a and b returns b if a is True, else returns a.
        package_name = get_package_name(folder_path) or package_name
        top_package_name = top_package_name or (package_name and os.path.dirname(folder_path)) 
        process_python_files(top_package_name, folder_path, package_name=package_name)
        
        parent_folder_name = os.path.basename(os.path.dirname(folder_path))
        for file_name in os.listdir(folder_path):
            process_file_factory(folder_path, file_name, top_package_name, package_name, parent_folder_name)
                
           

In [12]:

# Example usage
process_python_files(None, "/home/ruzickal/Code/Fingerprints")

Project: DataView: Processing DataView in folder src: /home/ruzickal/Code/Fingerprints/DataView/src/DataView/Caching.py - None - a0e5dae1427e11602611dea91b71a0d5
Project: DataView: Processing DataView in folder src: /home/ruzickal/Code/Fingerprints/DataView/src/DataView/__init__.py - None - d41d8cd98f00b204e9800998ecf8427e
Project: DataView: Processing DataView in folder src: /home/ruzickal/Code/Fingerprints/DataView/src/DataView/DataView.py - None - 3f334887cfefd7727fb777879d920c06
Project: DataView: Processing DataView in folder src: /home/ruzickal/Code/Fingerprints/DataView/src/DataView/Application.py - None - 39cbdffdf99828d4253ff7e7b2e60ff1
ResourcesDict(None, {'no_promt': True, 'metadata': ResourcesDict(None, {'name': 'Notebook'}), 'output_extension': '.py', 'deprecated': <function deprecated at 0x7fdb2bd663e0>, 'raw_mimetypes': ['text/x-python', ''], 'global_content_filter': {'include_code': True, 'include_markdown': True, 'include_raw': True, 'include_unknown': True, 'include_i

KeyboardInterrupt: 